In [1]:
import sys
sys.path.append("../")

In [2]:
import glob

path_dir_test_data = "../test_data/html"
list_path_html = glob.glob(path_dir_test_data + "/*.html")
len(list_path_html)

10

In [3]:
from langchain.document_loaders import BSHTMLLoader, UnstructuredHTMLLoader

loader = BSHTMLLoader(list_path_html[0], "utf-8")
loaded = loader.load()  # -> List[Document]

In [4]:
type(loaded[0].page_content)

str

In [5]:
from docmesh.embedding import LangchainOpenAIEmbeddingModel
from docmesh.llm import LangchainOpenAILLM
from docmesh.config import Config

from dotenv import load_dotenv

load_dotenv()


generator_llm = LangchainOpenAILLM(model=Config.LLM_MODEL)
critic_llm = LangchainOpenAILLM(model=Config.LLM_MODEL)
embedding = LangchainOpenAIEmbeddingModel()

In [6]:
from docmesh.evaluation import RAGTestsetGenerator, load_personas_from_yaml

list_persona = load_personas_from_yaml('persona.yaml')
list_persona

[Persona(name='Junior Developer', role_description='A developer who has just started their career in software development. They have basic programming knowledge and are actively learning how to collaborate on small tasks or bug fixes.'),
 Persona(name='Intermediate Developer', role_description='A developer with around 2–5 years of professional experience. They can independently handle most tasks using common frameworks and libraries, have a reasonable grasp of code quality and design patterns, and can mentor junior developers.'),
 Persona(name='Senior Developer', role_description='A seasoned developer with 5 or more years of hands-on experience. They are proficient in large-scale system design, performance optimization, and can effectively lead teams or mentor other developers.'),
 Persona(name='University Student', role_description='A student majoring in computer science or a related field, learning fundamental programming skills and theoretical knowledge. They may lack practical expe

In [7]:
generator = RAGTestsetGenerator(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    generator_embeddings=embedding,
    list_persona=list_persona
                               )

In [8]:
result = generator.create_testset(list_path_html, testset_size=len(list_path_html))
result_pandas = result.to_pandas()

Applying HeadlinesExtractor:   0%|          | 0/9 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/10 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/12 [00:00<?, ?it/s]

Property 'summary' already exists in node 'e862ea'. Skipping!
Property 'summary' already exists in node '8168c7'. Skipping!
Property 'summary' already exists in node '3f93c9'. Skipping!


...

unable to apply transformation: 'ChatOpenAI' object has no attribute 'embed_query'
unable to apply transformation: 'ChatOpenAI' object has no attribute 'embed_query'
unable to apply transformation: 'ChatOpenAI' object has no attribute 'embed_query'
unable to apply transformation: 'ChatOpenAI' object has no attribute 'embed_query'
unable to apply transformation: 'ChatOpenAI' object has no attribute 'embed_query'
unable to apply transformation: 'ChatOpenAI' object has no attribute 'embed_query'
unable to apply transformation: 'ChatOpenAI' object has no attribute 'embed_query'
unable to apply transformation: 'ChatOpenAI' object has no attribute 'embed_query'
unable to apply transformation: 'ChatOpenAI' object has no attribute 'embed_query'
unable to apply transformation: 'ChatOpenAI' object has no attribute 'embed_query'
unable to apply transformation: 'ChatOpenAI' object has no attribute 'embed_query'
unable to apply transformation: 'ChatOpenAI' object has no attribute 'embed_query'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

unable to apply transformation: Node 4e11857d-487d-4b23-8de8-b7a696474a78 has no summary_embedding


Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
result_pandas

,user_input,reference_contexts,reference,synthesizer_name
0,What is LangChain and its significance in the ...,[Introduction | 🦜️🔗 LangChain Skip to main con...,LangChain is a platform that offers various to...,single_hop_specifc_query_synthesizer
1,How can a Junior Developer utilize CustomModel...,[How-to guidesHow-to guidesHow to use tools in...,To utilize CustomModels in software developmen...,single_hop_specifc_query_synthesizer
2,What is Pydantic and how does it relate to sof...,[IntegrationsAPI ReferenceMoreContributingPeop...,Pydantic is a compatibility tool that helps de...,single_hop_specifc_query_synthesizer
3,What is NVIDIA's role in the LangChain framework?,"[monitor and evaluate your applications, so th...",NVIDIA is one of the chat model providers inte...,single_hop_specifc_query_synthesizer
4,What is LangChain and how does it relate to th...,[Providers | 🦜️🔗 LangChain Skip to main conten...,LangChain is hosting the Interrupt: The Agent ...,single_hop_specifc_query_synthesizer
5,How can a Junior Developer migrate from v0.0 c...,[<1-hop>\n\nHow-to guidesHow-to guidesHow to u...,A Junior Developer can migrate from v0.0 chain...,multi_hop_specific_query_synthesizer
6,What is the advanced implementation with LangG...,"[<1-hop>\n\nBaseChatMessageHistory, you do not...",The advanced implementation with LangGraph inv...,multi_hop_specific_query_synthesizer
7,How can a Junior Developer migrate from v0.0 c...,[<1-hop>\n\nFew-shot promptingConceptual guide...,A Junior Developer can migrate from v0.0 chain...,multi_hop_specific_query_synthesizer
8,How can an Intermediate Developer build a simp...,[<1-hop>\n\nIntroduction | 🦜️🔗 LangChain Skip ...,An Intermediate Developer can build a simple L...,multi_hop_specific_query_synthesizer
9,How can a Junior Developer set up a connection...,"[<1-hop>\n\nnatural language answer. First, we...",A Junior Developer can set up a connection wit...,multi_hop_specific_query_synthesizer


In [10]:
result_pandas.to_csv("ragas_synthetic_dataset.csv", index=False)